In [1]:
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
ps = PorterStemmer()

In [3]:
file = r'.\Data\train\train.json'
with open(file) as train_file:
    dict_train = json.load(train_file)

In [4]:
sno = []
cuisine = []
ingredients = []
for i in range(len(dict_train)):
    sno.append(dict_train[i]['id'])
    cuisine.append(dict_train[i]['cuisine'])
    ingredients.append(dict_train[i]['ingredients'])

In [5]:
import pandas as pd
df = pd.DataFrame({'id':sno, 
                   'cuisine':cuisine, 
                   'ingredients':ingredients})

In [6]:
def analyzer(df):
    new_df = []
    for s in df:
        s = ' '.join(s)
        new_df.append(s) 
    import re
    mod_df=[]
    for s in new_df:
        s=re.sub(r'[^\w\s]','',s)
        s=re.sub(r"(\d)", "", s)    
        s=re.sub(r'\([^)]*\)', '', s)
        s=re.sub(u'\w*\u2122', '', s)
        s=s.lower()    
        #Remove Stop Words
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(s)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        s=' '.join(filtered_sentence)    
        #Porter Stemmer Algorithm
        words = word_tokenize(s)
        word_ps=[]
        for w in words:
           word_ps.append(ps.stem(w))
        s=' '.join(word_ps)
        mod_df.append(s)
    return mod_df

In [7]:
def analyzer1(df):
    import re
    mod_df=[]
    for s in df:
        s=re.sub(r'[^\w\s]','',s)
        s=re.sub(r"(\d)", "", s)
        s=re.sub(r'\([^)]*\)', '', s)   
        s=re.sub(u'\w*\u2122', '', s)
        s=s.lower()
        #Remove Stop Words
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(s)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        s=' '.join(filtered_sentence)
        #Porter Stemmer Algorithm
        words = word_tokenize(s)
        word_ps=[]
        for w in words:
           word_ps.append(ps.stem(w))
        s=' '.join(word_ps)
        mod_df.append(s)
    return mod_df

In [8]:
df['ingredients_mod']= analyzer(df['ingredients'])

In [9]:
mylist =['fish', 'goat', 'chicken','beef','pork','prawn','egg','Katsuobushi','mackrel','fillet','lamb','steak','salmon','shrimp']
pattern = '|'.join(mylist)
df['veg']=df.ingredients_mod.str.contains(pattern) 
df.loc[df.veg == True,'veg'] = 'non-vegetarian'
df.loc[df.veg == False,'veg'] = 'vegetarian'

In [10]:
import pandas as pd
#import re
file = r'.\Data\test\test.csv'
df_test = pd.read_csv(file,encoding='ISO-8859-1')
df_test['ingredients_mod']= analyzer1(df_test['ingredients'])

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
ingredient_train = vectorizer.fit_transform(df['ingredients_mod'])
ingredient_test = vectorizer.transform(df_test['ingredients_mod'])

In [12]:
from sklearn import preprocessing
elable = preprocessing.LabelEncoder()
elable.fit(df['cuisine'])
df['cuisine']=elable.transform(df['cuisine'])
elable.fit(df['veg'])
df['veg']=elable.transform(df['veg'])

In [13]:
veg_map={'0':'non-vegetarian', '1':'vegetarian'}
cuisine_map={'0':'brazilian', '1':'british', '2':'cajun_creole', '3':'chinese', '4':'filipino', '5':'french', '6':'greek', '7':'indian', '8':'irish', '9':'italian', '10':'jamaican', '11':'japanese', '12':'korean', '13':'mexican', '14':'moroccan', '15':'russian', '16':'southern_us', '17':'spanish', '18':'thai', '19':'vietnamese'}

In [14]:
Cuisine_train=[]
Type_train=[]
Cuisine_train = df['cuisine']
Type_train = df['veg']

In [15]:
from sklearn import svm
lin_cuisine = svm.LinearSVC(C=1)
lin_veg = svm.LinearSVC(C=1)
lin_cuisine.fit(ingredient_train, Cuisine_train)
lin_veg.fit(ingredient_train, Type_train)
Cuisine_pred_idx=lin_cuisine.predict(ingredient_test)
Type_pred_idx=lin_veg.predict(ingredient_test)

In [23]:
Cuisine_pred=[]
for i in range(len(Cuisine_pred_idx)):
    Cuisine_pred.append(cuisine_map[str(Cuisine_pred_idx[i])])
       
Type_pred=[]
for i in range(len(Type_pred_idx)):
    Type_pred.append(veg_map[str(Type_pred_idx[i])])    

df_test['cuisine'] =  Cuisine_pred
df_test['Veg/NonVeg'] =  Type_pred
Result = pd.DataFrame({'Actual Cuisine':df_test['receipes'], 'Predicted Cuisine':Cuisine_pred, 'Veg/NonVeg':Type_pred})
print(Result)

                                       Actual Cuisine Predicted Cuisine  \
0                                   American pancakes       southern_us   
1              Bourbon, black cherry & bacon brownies       southern_us   
2                   Safe-to-eat uncooked cookie dough       southern_us   
3   Tomato, avocado & corn salad with migas & butt...           mexican   
4                                        Creamed corn           mexican   
5                                         Pumpkin pie       southern_us   
6                            American burnt onion dip           mexican   
7                               Smoky braised brisket          moroccan   
8                  Fully loaded Cajun chicken burgers          moroccan   
9                                         S'mores dip            french   
10                                   Texas BBQ medley       southern_us   
11                                All-American T-bone           italian   
12                       

In [26]:
df_test.to_csv("output.csv", index=False)